In the first cell, we define our query function. This is to be copied and pasted each time we want to query science windows. 

In [ ]:
import time
import astroquery.heasarc
from astropy.coordinates import SkyCoord
import astropy.units as u

Heasarc = astroquery.heasarc.Heasarc()

def get_scw_list(ra_obj, dec_obj,radius,start_date,end_date ):
    R = Heasarc.query_region(
            position = SkyCoord(ra_obj, dec_obj, unit='deg'),
            radius = f"{radius} deg",
            catalog = 'intscw', # 'mission' has been deprecated, use catalog instead
            time = start_date + " .. " + end_date,
            good_isgri = ">1000",
        )
    R.sort('scw_id') # changed SCW_ID to scw_id

    return R['scw_id'], R['scw_ver']  # changed SCW_VER to scw_ver


assert astroquery.__version__ >= '0.4.2.dev6611'
assert 'isdc' in astroquery.heasarc.Conf.server.cfgtype

In this second cell, define our source or region to query.
What we want to do for Jupiter is find Jupiter's time-dependent position and query that for a given search radius and time interval. We use astroquery's JPL Horizons module.

In [17]:
from astroquery.jplhorizons import Horizons

epochs = {'start':'2004-01-01', 'stop':'2004-02-01', 'step':'1d'}

# Jupiter's ID in Horizons is 599
# We use @earth as we want geocentric coordinates
jupiter = Horizons(id='599', location='@0',epochs=epochs) # 599 is Jupiter's ID in this module
eph = jupiter.ephemerides()
print(eph)

ra_values = eph['RA']  # in degrees
dec_values = eph['DEC']  # in degrees
print(eph[['datetime_str', 'RA', 'DEC']])

  targetname     datetime_str   datetime_jd ... alpha_true  PABLon  PABLat
     ---             ---             d      ...    deg       deg     deg  
------------- ----------------- ----------- ... ---------- -------- ------
Jupiter (599) 2004-Jan-01 00:00   2453005.5 ...     0.0176 159.0596 1.1123
Jupiter (599) 2004-Jan-02 00:00   2453006.5 ...     0.0175 159.1366 1.1132
Jupiter (599) 2004-Jan-03 00:00   2453007.5 ...     0.0175 159.2135 1.1141
Jupiter (599) 2004-Jan-04 00:00   2453008.5 ...     0.0175 159.2904  1.115
Jupiter (599) 2004-Jan-05 00:00   2453009.5 ...     0.0175 159.3673 1.1159
Jupiter (599) 2004-Jan-06 00:00   2453010.5 ...     0.0175 159.4442 1.1168
Jupiter (599) 2004-Jan-07 00:00   2453011.5 ...     0.0174  159.521 1.1178
Jupiter (599) 2004-Jan-08 00:00   2453012.5 ...     0.0174 159.5979 1.1186
Jupiter (599) 2004-Jan-09 00:00   2453013.5 ...     0.0174 159.6748 1.1195
Jupiter (599) 2004-Jan-10 00:00   2453014.5 ...     0.0174 159.7516 1.1204
          ...            

Error due to seperation between INTEGRAL and Earth.

In [ ]:
D = 150e3 # max distance between INTEGRAL and Earth (~ 150 000 km)
r = 4 * 149597870.7 # min distance between Jupiter and Earth is ~ 4 AU 

error = D/r * 206265/60 # convert to arcmin
print("Angular error is ", error, " arcmin")

In [18]:
scw_ids = []
scw_versions = []

# Loop through the ephemeris
for row in eph:
    ra = float(row['RA'])
    dec = float(row['DEC'])
    radius = 20.0 # search radius in degrees
    scw_id, scw_ver = get_scw_list(ra, dec, radius, '2004-01-01T00:00:00', '2004-02-01T00:00:00')
    scw_ids.extend(scw_id)
    scw_versions.extend(scw_ver)

print("Found science windows:", scw_ids)


Found science windows: ['000700000061', '000700000071', '000752000010', '000752000020', '000752000030', '000752000040', '000752000050', '000752000060', '000752000070', '000752000080', '000752000090', '000752000100', '000752000110', '000752000120', '000752000130', '000752200010', '000752200020', '000752200030', '000752200040', '000752200051', '000752200060', '000752200071', '000752200080', '000752200091', '000752200100', '000752200111', '000752200120', '000752200131', '000752200140', '000752200151', '000752200160', '000752200171', '000752200180', '000752200190', '000752200200', '000800000012', '000800000020', '000800000030', '000800000040', '000800000050', '000800000060', '000853000010', '000853000020', '000853000030', '000853000040', '000853000050', '000853000060', '000853000100', '000854000010', '000854000020', '000854000030', '000854100010', '000854100020', '000854100030', '000854100040', '000855000010', '000855000020', '000855000030', '000855000040', '000855000050', '000855500010', 

We then pick 10 science windows at random. 

In [23]:
import random

nscw = 10
if len(scw_ids) == 0:
    raise Exception('No science windows')

scw_pick = [(s+"."+v).strip() for s,v in zip(scw_ids,scw_versions)]

if nscw >0 and len(scw_pick)>=nscw:
    random.seed(0)
    scw_pick = random.sample(scw_pick, nscw)
scw_pick = sorted(scw_pick)

print(scw_pick)

['001300010160.001', '100500300010.001', '111600710021.001', '118000560010.001', '146600660020.001', '155300030181.001', '176300390021.001', '189900390010.001', '236700140021.001', '284000010010.001']


Before moving on to using this query for our ODA, we need to input our token. 

In [ ]:
token = ''

import getpass
token = getpass.getpass('Insert the token')

{'sub': 'hucklethums@icloud.com',
 'email': 'hucklethums@icloud.com',
 'name': 'interstellxr',
 'roles': 'authenticated user, public-pool-hpc',
 'exp': 1741191208}

We also add logging to help visualize the process.

In [ ]:
import logging
logging.getLogger().setLevel(logging.INFO) # WARNING, INFO or DEBUG
logging.getLogger('oda_api').addHandler(logging.StreamHandler())

Now let us look at an example.

In [ ]:
from oda_api.api import DispatcherAPI

disp = DispatcherAPI(url="https://www.astro.unige.ch/mmoda/dispatch-data")

par_dict = {
"E1_keV": "15",
"E2_keV": "100",
"detection_threshold": "7",
"instrument": "isgri",
"osa_version": "OSA10.2",
"product": "isgri_image",
"product_type": "Real",
"scw_list": scw_pick,
'token': token
}
# "integral_data_rights": "all-private" had to be removed as didn't have the rights

data_collection = disp.get_product(**par_dict)
data_collection.show()
data_collection.mosaic_image_0_mosaic.show()
print(data_collection.mosaic_image_0_mosaic.show_meta())


please beware that by default, in a typical setup, oda_api will not output much. To learn how to increase the verbosity, please refer to the documentation: https://oda-api.readthedocs.io/en/latest/user_guide/ScienceWindowList.html?highlight=logging#Let's-get-some-logging . 
To disable this message you can pass `.get_product(..., silent=True)`


Finally, let us image our results.

In [ ]:
import matplotlib.pyplot as plt
from astropy.io import fits
import numpy as np

# from oda_api.plot_tools import OdaImage

fits_image = data_collection.mosaic_image_0_mosaic

with fits.open(fits_image) as hdul:
    image_data = hdul[0].data 

# Plot the image
plt.figure(figsize=(10, 8))
plt.imshow(image_data, cmap='inferno', origin='lower', norm=plt.Normalize(vmin=np.percentile(image_data, 5), vmax=np.percentile(image_data, 99)))  
plt.colorbar(label="Counts")
plt.title("INTEGRAL/ISGRI X-ray Image")
plt.xlabel("X Pixel")
plt.ylabel("Y Pixel")
plt.show()